# PyTorchで線形変換WXを実現する2つの方法

PyTorchで線形変換$WX$を実現する方法を考える.  
まず普通に$WX$を実装すると以下のようになる.

In [2]:
import torch

In [3]:
W = torch.tensor(
    [
        [1., 2., 3.], 
        [4., 5., 6.], 
        [7., 8., 9.]
    ]
)
X = torch.tensor(
    [
        [0.1, 0.2, 0.3], 
        [0.4, 0.5, 0.6], 
        [0.7, 0.8, 0.9]
    ]
)

print(W @ X)

tensor([[ 3.0000,  3.6000,  4.2000],
        [ 6.6000,  8.1000,  9.6000],
        [10.2000, 12.6000, 15.0000]])


これは$W$の行ベクトルと$X$の列ベクトルを個別にかけた場合も当然同じ結果になる.

In [7]:
for w in W:
    t = []
    for x in X.T:
        t.append(w @ x)
    print(t)

[tensor(3.0000), tensor(3.6000), tensor(4.2000)]
[tensor(6.6000), tensor(8.1000), tensor(9.6000)]
[tensor(10.2000), tensor(12.6000), tensor(15.)]


次に, PyTorchのLinearクラスを使って同じことをしてみる.

In [8]:
import torch.nn as nn

L = nn.Linear(3, 3, bias=False)
L.weight = nn.Parameter(
    torch.tensor(
        [
            [1., 2., 3.], 
            [4., 5., 6.], 
            [7., 8., 9.], 
        ]
    )
)
print(L(X))

tensor([[ 1.4000,  3.2000,  5.0000],
        [ 3.2000,  7.7000, 12.2000],
        [ 5.0000, 12.2000, 19.4000]], grad_fn=<MmBackward0>)


すると上のように,W@Xを実行したのとは別の結果が出力される.  
これはPyTorchのLinearクラスの関数が$WX$ではなく$XW^{\top}$として実装されているためで,Linearオブジェクトを使って$WX$を実現したい場合は関数実行の際に$X$を転置して与え,出力結果をさらに転置する($(AB)^{\top} = B^{\top} A^{\top}$のため,$(X^{\top} W^{\top})^{\top} = WX$).  

In [119]:
L = nn.Linear(3, 3, bias=False)
L.weight = nn.Parameter(
    torch.tensor(
        [
            [1., 2., 3.], 
            [4., 5., 6.], 
            [7., 8., 9.], 
        ]
    )
)
print(L(X.T).T)

tensor([[ 3.0000,  3.6000,  4.2000],
        [ 6.6000,  8.1000,  9.6000],
        [10.2000, 12.6000, 15.0000]], grad_fn=<PermuteBackward0>)


同様に,$W$が$N \times M$行列の場合も以下のようになる.

In [122]:
X = torch.tensor(
    [
        [0.1, 0.2, 0.3], 
        [0.4, 0.5, 0.6], 
        [0.7, 0.8, 0.9], 
        [1.0, 1.1, 1.2]
    ]
)

L = nn.Linear(3, 3, bias=False)
L.weight = nn.Parameter(
    torch.tensor(
        [
            [1., 2., 3., 4.], 
            [5., 6., 7., 8.], 
            [9., 10., 11., 12.]
        ]
    )
)
print(L(X.T).T)

tensor([[ 7.0000,  8.0000,  9.0000],
        [15.8000, 18.4000, 21.0000],
        [24.6000, 28.8000, 33.0000]], grad_fn=<PermuteBackward0>)


In [124]:
W = torch.tensor(
    [
        [1., 2., 3., 4.], 
        [5., 6., 7., 8.], 
        [9., 10., 11., 12.]
    ]
)

print(W @ X)

tensor([[ 7.0000,  8.0000,  9.0000],
        [15.8000, 18.4000, 21.0000],
        [24.6000, 28.8000, 33.0000]])


ただ,通常入力データ$X$は行ベクトルを1サンプルとするので$WX$として実装するよりは$XW^{\top}$とする方が一般的ではある.  
これは重みパラメータ$W$を$X$の前にもってくるとサンプル数を固定長にするか$X$を転置するかしなければならなくなるからでもある.  
つまり,$W$と$X$は$[N \times M]$ @ $[M \times N]$の形にならなければならないが,$X$のサンプル数を1つ増やした場合$X$の形は$[(M + 1) \times N]$になってしまい,そのまま実行するとバグる.  
それを回避するために$W$を$[M \times N]$の形に設定した上で$L(X^{\top})$として$X$を転置して与えるという方法もあるが,実行するたびに$X$を転置するのは操作性としてはよくない.  
そのため,あえてLinearオブジェクトを使って$WX$を実現する実用的な意味はない.  
今回の実装はあくまでPyTorchのLinearオブジェクトの挙動を確認するための実験.